# Task 5: Strategy Backtesting

## Objective
Validate the optimized portfolio strategy by simulating performance on a holdout period and comparing it against a benchmark (60% SPY / 40% BND).

## Deliverables
- Cumulative returns comparison plot (strategy vs benchmark)
- Performance metrics table (total return, annualized return, Sharpe ratio, max drawdown)
- Written conclusion on strategy viability


## 1. Imports and Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', '{:.4f}'.format)

TRADING_DAYS = 252
RISK_FREE_RATE = 0.02  # 2% annual risk-free rate

print('Setup complete!')

## 2. Load Data and Portfolio Recommendation

In [ ]:
# Load adjusted close prices
prices = pd.read_csv('../data/processed/adj_close_prices.csv', parse_dates=['Date'], index_col='Date')
prices = prices.sort_index()

tickers = ['TSLA', 'BND', 'SPY']
prices = prices[tickers].dropna()

print(f'Price data: {prices.index.min().date()} to {prices.index.max().date()}')
print(f'Shape: {prices.shape}')
prices.tail()

In [ ]:
# Load portfolio recommendation from Task 4
portfolio_rec = joblib.load('../data/processed/models/portfolio_recommendation.joblib')

strategy_weights = pd.Series(portfolio_rec['weights']).reindex(tickers).fillna(0.0)
strategy_weights = strategy_weights / strategy_weights.sum()

print('Strategy weights:')
print(strategy_weights)

In [ ]:
# Benchmark weights (60% SPY / 40% BND)
benchmark_weights = pd.Series({'TSLA': 0.0, 'BND': 0.4, 'SPY': 0.6})
benchmark_weights = benchmark_weights.reindex(tickers).fillna(0.0)

print('Benchmark weights:')
print(benchmark_weights)

## 3. Define Backtest Window

Use the last year of data (January 2025 to January 2026).

In [ ]:
start_date = '2025-01-01'
end_date = '2026-01-15'

backtest_prices = prices.loc[start_date:end_date].copy()

print(f'Backtest window: {backtest_prices.index.min().date()} to {backtest_prices.index.max().date()}')
print(f'Backtest observations: {len(backtest_prices)}')

## 4. Backtest Functions

In [ ]:
def compute_drawdown(equity_curve):
    rolling_max = equity_curve.cummax()
    drawdown = equity_curve / rolling_max - 1
    return drawdown

def compute_metrics(daily_returns, equity_curve, risk_free_rate=0.02):
    total_return = equity_curve.iloc[-1] - 1
    annual_return = equity_curve.iloc[-1] ** (TRADING_DAYS / len(daily_returns)) - 1
    annual_volatility = daily_returns.std() * np.sqrt(TRADING_DAYS)
    sharpe = (daily_returns.mean() * TRADING_DAYS - risk_free_rate) / annual_volatility if annual_volatility > 0 else np.nan
    max_drawdown = compute_drawdown(equity_curve).min()

    return {
        'Total Return': total_return,
        'Annualized Return': annual_return,
        'Annualized Volatility': annual_volatility,
        'Sharpe Ratio': sharpe,
        'Max Drawdown': max_drawdown
    }

def backtest_portfolio(price_df, target_weights, rebalance=None):
    """
    price_df: DataFrame of prices (columns = tickers)
    target_weights: Series of weights (sum to 1)
    rebalance: None (buy & hold) or 'M' (monthly)
    """
    price_df = price_df.copy()
    tickers = price_df.columns

    target_weights = target_weights.reindex(tickers).fillna(0.0)
    target_weights = target_weights / target_weights.sum()

    dates = price_df.index
    equity_curve = pd.Series(index=dates, dtype=float)

    # Initial allocation
    initial_value = 1.0
    holdings = (initial_value * target_weights) / price_df.iloc[0]
    equity_curve.iloc[0] = initial_value

    # Determine rebalance dates (first trading day of each month)
    if rebalance == 'M':
        rebalance_dates = price_df.groupby(price_df.index.to_period('M')).head(1).index
        rebalance_dates = set(rebalance_dates[1:])  # exclude the first day
    else:
        rebalance_dates = set()

    for i, date in enumerate(dates[1:], start=1):
        # Update portfolio value
        value = float(np.sum(holdings * price_df.loc[date]))
        equity_curve.iloc[i] = value

        # Rebalance if required
        if date in rebalance_dates:
            holdings = (value * target_weights) / price_df.loc[date]

    daily_returns = equity_curve.pct_change().fillna(0.0)
    return equity_curve, daily_returns

## 5. Run Backtest

Set `REBALANCE = 'M'` for monthly rebalancing or `None` for buy-and-hold.

In [ ]:
REBALANCE = 'M'  # Options: 'M' or None

strategy_equity, strategy_returns = backtest_portfolio(
    backtest_prices, strategy_weights, rebalance=REBALANCE
)

benchmark_equity, benchmark_returns = backtest_portfolio(
    backtest_prices, benchmark_weights, rebalance=REBALANCE
)

print('Backtest completed!')

## 6. Performance Metrics

In [ ]:
strategy_metrics = compute_metrics(strategy_returns, strategy_equity, RISK_FREE_RATE)
benchmark_metrics = compute_metrics(benchmark_returns, benchmark_equity, RISK_FREE_RATE)

metrics_df = pd.DataFrame({
    'Strategy': strategy_metrics,
    'Benchmark (60/40)': benchmark_metrics
})

print('=' * 60)
print('BACKTEST PERFORMANCE METRICS')
print('=' * 60)
metrics_df

## 7. Plot Cumulative Returns

In [ ]:
# Normalize equity curves to start at 1
strategy_curve = strategy_equity / strategy_equity.iloc[0]
benchmark_curve = benchmark_equity / benchmark_equity.iloc[0]

plt.figure(figsize=(14, 6))
plt.plot(strategy_curve.index, strategy_curve, label='Strategy (Optimized Portfolio)', linewidth=2)
plt.plot(benchmark_curve.index, benchmark_curve, label='Benchmark (60% SPY / 40% BND)', linewidth=2)

plt.title('Backtest Cumulative Returns: Strategy vs Benchmark', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return (Normalized)', fontsize=12)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/backtest_strategy_vs_benchmark.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Conclusions and Reflection

In [ ]:
strategy_outperformed = strategy_metrics['Total Return'] > benchmark_metrics['Total Return']

conclusion = f"""
BACKTEST CONCLUSION (1-2 paragraphs)
{'=' * 60}

Over the backtest period ({backtest_prices.index.min().date()} to {backtest_prices.index.max().date()}),
the optimized portfolio {'outperformed' if strategy_outperformed else 'underperformed'} the benchmark
60% SPY / 40% BND strategy. The optimized portfolio delivered a total return of
{strategy_metrics['Total Return']*100:.2f}%, compared to {benchmark_metrics['Total Return']*100:.2f}%
for the benchmark. Risk-adjusted performance, as measured by the Sharpe ratio,
was {strategy_metrics['Sharpe Ratio']:.3f} for the strategy vs {benchmark_metrics['Sharpe Ratio']:.3f}
for the benchmark.

These results suggest that incorporating forecast-driven expected returns can add
value in portfolio construction, especially when combined with diversification
assets like BND and SPY. However, this backtest is limited to a single one-year
period and does not include transaction costs, taxes, or slippage. The results
should therefore be viewed as indicative rather than definitive. A longer backtest
across multiple market regimes would be required to validate robustness.
"""

print(conclusion)

## 9. Save Outputs

In [ ]:
# Save equity curves and metrics
equity_df = pd.DataFrame({
    'Strategy': strategy_curve,
    'Benchmark': benchmark_curve
})
equity_df.to_csv('../data/processed/backtest_equity_curves.csv')

metrics_df.to_csv('../data/processed/backtest_metrics.csv')

print('Outputs saved:')
print('  - data/processed/backtest_strategy_vs_benchmark.png')
print('  - data/processed/backtest_equity_curves.csv')
print('  - data/processed/backtest_metrics.csv')

In [ ]:
print('\n' + '=' * 60)
print('TASK 5 COMPLETE')
print('=' * 60)
print('\nDeliverables:')
print('  ✓ Cumulative returns comparison plot')
print('  ✓ Performance metrics table')
print('  ✓ Written conclusion on strategy viability')
print('\nSaved outputs:')
print('  - data/processed/backtest_strategy_vs_benchmark.png')
print('  - data/processed/backtest_equity_curves.csv')
print('  - data/processed/backtest_metrics.csv')